# 1.Importing Libraries

In [2]:
import os

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split

# 2. Reading the Data

In [9]:
project_dir = r'C:\Users\HP\Desktop\ML\Flight Price prediction sagemaker'
data_dir = r'Data'

In [12]:
def read_data(file_name):
    file_name = f'{file_name}.csv'
    file_path = os.path.join(project_dir,data_dir,file_name)

    return pd.read_csv(file_path)

In [13]:
flights = read_data("flight_price")
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [14]:
flights.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


# 3. Preliminary Analysis
## 3.1 Check Data Types

In [15]:
flights.dtypes


Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [16]:
value = flights.Date_of_Journey.iloc[6]
value

'12/03/2019'

In [17]:
value = flights.Dep_Time.iloc[6]
value

'18:55'

## 3.2 Check for Duplicates


In [18]:
flights.duplicated().sum()


np.int64(220)

In [19]:
(
	flights
	.loc[flights.duplicated(keep=False)]
	.sort_values(["Airline", "Date_of_Journey", "Source", "Destination"])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


## 3.3 Observations

The type of Date_of_Journey, Dep_Time and Arrival_Time should be changed to datetime

The type of Duration and Total_Stops is mixed. It should be numeric type.

There're 220 duplicates. These should be removed.

## 4. Detailed Analysis
### Airline

In [20]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [26]:
flights['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [23]:
flights.Airline.str.replace(" Premium economy", "").str.replace(" Business", "").str.title()

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [25]:
(
    flights
	.Airline
	.str.replace(" Premium economy", "")
	.str.replace(" Business", "")
	.str.title()
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

## Date_of_Journey

### dep_time

In [42]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [43]:
(
    flights
    .Dep_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [45]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\HP\AppData\Local\Temp\ipykernel_19460\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival Time

In [57]:
(
    flights
    .Arrival_Time
    .loc[lambda ser : ser.str.contains("[^0:9]")]
    .str.split(" ",n=1)
    .str.get(1)
    .unique()
)

array(['22 Mar', nan, '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### Duration

In [58]:
(
	flights
	.Duration
	.loc[lambda ser: ~ser.str.contains("m")]
	.unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [59]:
(
	flights
	.Duration
	.loc[lambda ser: ~ser.str.contains("h")]
	# .unique()
)

6474    5m
Name: Duration, dtype: object

In [63]:
def convert_to_minutes(ser):
	return (
		ser
		.str.split(" ", expand=True)
		.set_axis(["hour", "minute"], axis=1)
		.assign(
			hour=lambda df_: (
				df_
				.hour
				.str.replace("h", "")
				.astype(int)
				.mul(60)
			),
			minute=lambda df_: (
				df_
				.minute
				.str.replace("m", "")
				.fillna("0")
				.astype(int)
			)
		)
		.sum(axis=1)
	)

## 4 Detailed Analysis

## 5. Cleaning

In [64]:
def clean_data(df):
    return (
        df
	    .drop(index=6474)
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include='O').columns
        })
        .rename(columns=str.lower)
        .assign(
			airline=lambda df_: (
				df_
				.airline
				.str.replace(" Premium economy", "")
				.str.replace(" Business", "")
				.str.title()
			),
            date_of_journey=lambda df_: pd.to_datetime(df_.date_of_journey, dayfirst=True),
			dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
			arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_: (
				df_
				.total_stops
				.replace("non-stop", "0")
				.str.replace(" stops?", "", regex=True)
				.pipe(lambda ser: pd.to_numeric(ser))
			),
            additional_info=lambda df_: df_.additional_info.replace("No info", "No Info")


        )
        .drop(columns = 'route')
    )

In [68]:
clean_flights = clean_data(flights)

C:\Users\HP\AppData\Local\Temp\ipykernel_19460\2053134854.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
C:\Users\HP\AppData\Local\Temp\ipykernel_19460\2053134854.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,


In [69]:
X = clean_flights.drop(columns="price")
y = clean_flights.price.copy()

In [72]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6836, 9) (6836,)
(1709, 9) (1709,)
(2137, 9) (2137,)


In [75]:
def export_data(X, y, name):
	file_name = f"{name}.csv"
	file_path = os.path.join(project_dir, data_dir, file_name)

	X.join(y).to_csv(file_path, index=False)

	return pd.read_csv(file_path).head()

In [76]:
export_data(X_train, y_train, "train")


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-03,Delhi,Cochin,02:15:00,04:25:00,1570,1.0,No info,17024
1,Vistara,2019-03-24,Kolkata,Banglore,07:10:00,18:45:00,695,1.0,No info,16932
2,Spicejet,2019-04-09,Banglore,Delhi,09:30:00,12:20:00,170,0.0,No info,4423
3,Indigo,2019-04-27,Banglore,Delhi,21:15:00,00:15:00,180,0.0,No info,3943
4,Air India,2019-06-12,Delhi,Cochin,09:45:00,09:25:00,1420,1.0,No info,7480


In [77]:
export_data(X_val, y_val, "val")


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Kolkata,Banglore,16:30:00,12:00:00,1170,1.0,In-flight meal not included,8529
1,Jet Airways,2019-05-18,Banglore,Delhi,11:10:00,14:05:00,175,0.0,In-flight meal not included,5198
2,Multiple Carriers,2019-06-12,Delhi,Cochin,08:45:00,19:00:00,615,1.0,No info,11789
3,Multiple Carriers,2019-04-24,Delhi,Cochin,07:10:00,16:10:00,540,1.0,In-flight meal not included,6093
4,Jet Airways,2019-03-27,Delhi,Cochin,15:05:00,04:25:00,800,1.0,No info,12242


In [78]:
export_data(X_test, y_test, "test")


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-05-21,Delhi,Cochin,15:05:00,01:30:00,625,2.0,No info,16655
1,Goair,2019-06-03,Delhi,Cochin,10:35:00,19:35:00,540,1.0,No info,4959
2,Vistara,2019-05-09,Kolkata,Banglore,20:20:00,09:05:00,765,1.0,No info,9187
3,Indigo,2019-05-24,Chennai,Kolkata,14:45:00,17:05:00,140,0.0,No info,3858
4,Jet Airways,2019-05-21,Delhi,Cochin,22:50:00,04:25:00,335,1.0,In-flight meal not included,12898
